In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import keras
import numpy as np
import pandas as pd 

from keras.models import Model, clone_model
# from keras.models import load_model
from keras.layers import Dense, Input, Activation, Flatten, Add
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

from keras import backend
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [2]:
X, y, X_test = np.load('X.npy'), np.load('Y.npy'), np.load('X_test.npy')
X = np.expand_dims(X, axis=3).astype('float32') / 255.0
X_test = np.expand_dims(X_test, axis=3).astype('float32') / 255.0
y_cat = np_utils.to_categorical(y)

#shuffling
s = np.random.shuffle(np.arange(X.shape[0]))
X = X[s]
y_cat = y_cat[s]

In [ ]:
def createModel():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nClasses, activation='softmax'))
     
    return model

In [3]:
#modified resnet (could not run original resnet architecture on my computer)
def resnet(img_dim, output):
    def add(a,b):
        shape1 = backend.int_shape(a)
        shape2 = backend.int_shape(b)
        w = int(round(shape1[1]/shape2[1]))
        h = int(round(shape1[2]/shape2[2]))
        eq = shape1[3] == shape2[3]

        tmp = a
        if w>1 or h>1 or not eq:
            tmp = Conv2D(filters=shape2[3],kernel_size=(1,1),strides=(w,h),padding='valid',kernel_initializer='he_normal', kernel_regularizer=l2(regval))(a)
        return Add()([tmp, b])

    def layer(num_filt, size, strides, inp):
        tmp = BatchNormalization(axis=3)(inp)
        tmp = Activation('relu')(tmp)
        tmp = Conv2D(num_filt, size, strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(regval))(tmp)
        return tmp
    
    regval = 0.001
    l = Input(img_dim)

    x = Conv2D(128, (7,7), strides=2, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(regval))(l)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)

    num = 16
    xtmp = Conv2D(num, (1,1), strides=1, 
                  padding='same', 
                  kernel_initializer='he_normal', 
                  kernel_regularizer=l2(regval))(x)
    xtmp = layer(num, (3,3), 1, xtmp)
    xtmp = layer(num*4, (1,1), 1, xtmp)
    x = add(x,xtmp)

    xtmp = layer(num, (1,1), 1, x)
    xtmp = layer(num, (3,3), 1, xtmp)
    xtmp = layer(num*4, (1,1), 1, xtmp)
    x = add(x,xtmp)

    for i in xrange(4):
        xtmp = layer(num, (1,1), 1, x)
        xtmp = layer(num, (3,3), 1, xtmp)
        xtmp = layer(num*4, (1,1), 1, xtmp)
        x = add(x,xtmp)

        xtmp = layer(num, (1,1), 1, x)
        xtmp = layer(num, (3,3), 1, xtmp)
        xtmp = layer(num*4, (1,1), 1, xtmp)
        x = add(x,xtmp)
        x = layer(num, (1,1), 2, x)

        num*=2

    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    xshape = backend.int_shape(x)
    x = AveragePooling2D(pool_size=(xshape[1],xshape[2]), strides=(1,1))(x)
    x = Flatten()(x)

    x = Dense(output, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=l2(regval))(x)
    model = Model(l,x)
    return model

model = resnet(X.shape[1:], y_cat.shape[1])
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 168, 308, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 84, 154, 128) 6400        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 84, 154, 128) 512         conv2d_1[0][0]                   
________________________________________________________________________________________

In [4]:
#ensemble
def train_model(model, batch_size, epochs, x, y, test, kf, continue_training):
    n_fold = kf.get_n_splits()
    preds_test = np.zeros((len(test),y_cat.shape[-1]), dtype = np.float)
    
    models = []
    results = []
    i = 1

    for train_index, test_index in kf.split(x):
        x_train = x[train_index];
        x_valid = x[test_index]
        y_train = y[train_index];
        y_valid = y[test_index]

        traingen = ImageDataGenerator(
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')
        traingen.fit(x_train)

        validgen = ImageDataGenerator()
        validgen.fit(x_valid)
        
        filepath = 'mod_resnet.fold_{}.hdf5'.format(i)
        early = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=1e-4, mode='auto')
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')
        lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=1, patience=4, min_lr=1e-7)
        
        callbacks=[checkpoint, early, lr_reducer]

        train_steps = len(x_train) / batch_size
        valid_steps = len(x_valid) / batch_size
        test_steps = len(test) / batch_size

        model_k = model
        
        if continue_training:
            model_k.load_weights(filepath)

        model_k.compile(optimizer='adam', 
                      loss='categorical_crossentropy', 
                      metrics = ['accuracy'])
        
        history = model_k.fit_generator(traingen.flow(x_train, y_train, batch_size=batch_size),
                                      train_steps,
                                      epochs,
                                      verbose=1,
                                      callbacks=callbacks,
                                      validation_data=validgen.flow(x_valid, y_valid, batch_size=batch_size),
                                      validation_steps=valid_steps,
                                      shuffle=True)

        model_k.load_weights(filepath=filepath)
        
        preds_test_fold = model_k.predict(test, batch_size=batch_size, verbose=1)

        preds_test += preds_test_fold
        
        models.append(clone_model(model_k))
        results.append(preds_test_fold)

        print('\n\n')

        i += 1

        if i <= n_fold:
            print('Now beginning training for fold {}\n\n'.format(i))
        else:
            print('Finished training!')

    preds_test /= n_fold


    return preds_test, models, results

In [5]:
batch_size = 2**3
epoch_num = 200
kf = KFold(n_splits=5, shuffle=True)

test_pred, models, results = train_model(model, batch_size, epoch_num, X, y_cat, X_test, kf, True)

Epoch 1/200
66/66 [==============================] - 210s 3s/step - loss: 2.0342 - acc: 0.7102 - val_loss: 10.3500 - val_acc: 0.1875
Epoch 2/200
66/66 [==============================] - 209s 3s/step - loss: 1.9185 - acc: 0.7708 - val_loss: 8.9954 - val_acc: 0.2812
Epoch 3/200
66/66 [==============================] - 252s 4s/step - loss: 1.9270 - acc: 0.7614 - val_loss: 3.0573 - val_acc: 0.5078
Epoch 4/200
66/66 [==============================] - 233s 4s/step - loss: 1.8821 - acc: 0.7727 - val_loss: 3.6054 - val_acc: 0.5078
Epoch 5/200
66/66 [==============================] - 216s 3s/step - loss: 1.8238 - acc: 0.7879 - val_loss: 2.0943 - val_acc: 0.6797
Epoch 6/200
66/66 [==============================] - 206s 3s/step - loss: 1.7967 - acc: 0.7860 - val_loss: 2.7057 - val_acc: 0.6328
Epoch 7/200
66/66 [==============================] - 204s 3s/step - loss: 1.7568 - acc: 0.8163 - val_loss: 3.8081 - val_acc: 0.3750
Epoch 8/200
66/66 [==============================] - 204s 3s/step - loss: 1

Epoch 32/200
66/66 [==============================] - 195s 3s/step - loss: 1.1260 - acc: 0.9375 - val_loss: 1.0056 - val_acc: 0.9688
Epoch 33/200
66/66 [==============================] - 194s 3s/step - loss: 1.1440 - acc: 0.9337 - val_loss: 1.0927 - val_acc: 0.9375
Epoch 34/200
66/66 [==============================] - 196s 3s/step - loss: 1.1486 - acc: 0.9261 - val_loss: 1.0473 - val_acc: 0.9375
Epoch 35/200
66/66 [==============================] - 195s 3s/step - loss: 1.1423 - acc: 0.9337 - val_loss: 1.0683 - val_acc: 0.9297
Epoch 36/200
66/66 [==============================] - 195s 3s/step - loss: 1.1010 - acc: 0.9451 - val_loss: 1.0386 - val_acc: 0.9375
Epoch 37/200
66/66 [==============================] - 195s 3s/step - loss: 1.0945 - acc: 0.9508 - val_loss: 1.1112 - val_acc: 0.9062
Epoch 38/200
66/66 [==============================] - 195s 3s/step - loss: 1.0701 - acc: 0.9583 - val_loss: 0.9910 - val_acc: 0.9688
Epoch 39/200
66/66 [==============================] - 195s 3s/step - 

Epoch 64/200
66/66 [==============================] - 196s 3s/step - loss: 1.0531 - acc: 0.9602 - val_loss: 0.9537 - val_acc: 0.9688
Epoch 65/200
66/66 [==============================] - 196s 3s/step - loss: 1.0434 - acc: 0.9508 - val_loss: 0.9337 - val_acc: 0.9766
Epoch 66/200
66/66 [==============================] - 196s 3s/step - loss: 1.0353 - acc: 0.9583 - val_loss: 0.9541 - val_acc: 0.9688
Epoch 67/200
66/66 [==============================] - 196s 3s/step - loss: 1.0171 - acc: 0.9678 - val_loss: 0.9544 - val_acc: 0.9688
Epoch 68/200
66/66 [==============================] - 196s 3s/step - loss: 1.0527 - acc: 0.9470 - val_loss: 0.9500 - val_acc: 0.9688
Epoch 69/200
66/66 [==============================] - 195s 3s/step - loss: 1.0521 - acc: 0.9602 - val_loss: 0.9517 - val_acc: 0.9688
Epoch 70/200
66/66 [==============================] - 195s 3s/step - loss: 1.0685 - acc: 0.9489 - val_loss: 0.9393 - val_acc: 0.9766
Epoch 71/200
66/66 [==============================] - 195s 3s/step - 

66/66 [==============================] - 195s 3s/step - loss: 0.7699 - acc: 0.9640 - val_loss: 0.7092 - val_acc: 0.9766
Epoch 51/200
66/66 [==============================] - 195s 3s/step - loss: 0.7712 - acc: 0.9564 - val_loss: 0.7764 - val_acc: 0.9609
Epoch 52/200
66/66 [==============================] - 196s 3s/step - loss: 0.7889 - acc: 0.9527 - val_loss: 0.7283 - val_acc: 0.9609
Epoch 53/200
66/66 [==============================] - 196s 3s/step - loss: 0.7886 - acc: 0.9508 - val_loss: 0.7624 - val_acc: 0.9453
Epoch 54/200
66/66 [==============================] - 196s 3s/step - loss: 0.7792 - acc: 0.9583 - val_loss: 0.7664 - val_acc: 0.9531
Epoch 55/200
66/66 [==============================] - 206s 3s/step - loss: 0.7621 - acc: 0.9640 - val_loss: 0.7115 - val_acc: 0.9688
Epoch 56/200
66/66 [==============================] - 205s 3s/step - loss: 0.7592 - acc: 0.9659 - val_loss: 0.6967 - val_acc: 0.9766
Epoch 57/200
66/66 [==============================] - 205s 3s/step - loss: 0.7771 

Epoch 11/200
66/66 [==============================] - 207s 3s/step - loss: 1.3217 - acc: 0.8201 - val_loss: 2.2901 - val_acc: 0.6016
Epoch 12/200
66/66 [==============================] - 206s 3s/step - loss: 1.1670 - acc: 0.8655 - val_loss: 1.1341 - val_acc: 0.8359
Epoch 13/200
66/66 [==============================] - 204s 3s/step - loss: 1.0550 - acc: 0.9167 - val_loss: 0.9520 - val_acc: 0.9219
Epoch 14/200
66/66 [==============================] - 205s 3s/step - loss: 1.0339 - acc: 0.9186 - val_loss: 1.4857 - val_acc: 0.7734
Epoch 15/200
66/66 [==============================] - 204s 3s/step - loss: 1.0137 - acc: 0.9186 - val_loss: 1.0021 - val_acc: 0.8984
Epoch 16/200
66/66 [==============================] - 204s 3s/step - loss: 1.0506 - acc: 0.8920 - val_loss: 1.0011 - val_acc: 0.8984
Epoch 17/200
66/66 [==============================] - 204s 3s/step - loss: 0.9806 - acc: 0.9129 - val_loss: 1.6543 - val_acc: 0.7109
Epoch 18/200
66/66 [==============================] - 204s 3s/step - 

66/66 [==============================] - 206s 3s/step - loss: 0.8001 - acc: 0.9508 - val_loss: 0.6917 - val_acc: 0.9922
Epoch 74/200
66/66 [==============================] - 206s 3s/step - loss: 0.7683 - acc: 0.9678 - val_loss: 0.6929 - val_acc: 0.9922
Epoch 75/200
66/66 [==============================] - 206s 3s/step - loss: 0.7735 - acc: 0.9697 - val_loss: 0.6845 - val_acc: 0.9922
Epoch 76/200
66/66 [==============================] - 207s 3s/step - loss: 0.7891 - acc: 0.9659 - val_loss: 0.6889 - val_acc: 0.9922
Epoch 77/200
66/66 [==============================] - 206s 3s/step - loss: 0.7686 - acc: 0.9716 - val_loss: 0.6906 - val_acc: 0.9922
Epoch 00077: early stopping
300/300 [==============================] - 34s 113ms/step



Now beginning training for fold 4


Epoch 1/200
66/66 [==============================] - 210s 3s/step - loss: 1.2450 - acc: 0.7879 - val_loss: 2.4643 - val_acc: 0.5859
Epoch 2/200
66/66 [==============================] - 205s 3s/step - loss: 1.1021 - acc: 0.8

Epoch 9/200
66/66 [==============================] - 202s 3s/step - loss: 0.9073 - acc: 0.9053 - val_loss: 1.3502 - val_acc: 0.7500
Epoch 10/200
66/66 [==============================] - 203s 3s/step - loss: 0.8711 - acc: 0.9167 - val_loss: 0.9788 - val_acc: 0.8594
Epoch 11/200
66/66 [==============================] - 204s 3s/step - loss: 0.8317 - acc: 0.9280 - val_loss: 1.0381 - val_acc: 0.8281
Epoch 12/200
66/66 [==============================] - 204s 3s/step - loss: 0.8284 - acc: 0.9318 - val_loss: 0.8259 - val_acc: 0.9141
Epoch 13/200
66/66 [==============================] - 204s 3s/step - loss: 0.8221 - acc: 0.9223 - val_loss: 3.8507 - val_acc: 0.6172
Epoch 14/200
66/66 [==============================] - 203s 3s/step - loss: 0.8039 - acc: 0.9261 - val_loss: 0.7426 - val_acc: 0.9531
Epoch 15/200
66/66 [==============================] - 204s 3s/step - loss: 0.7936 - acc: 0.9280 - val_loss: 0.8539 - val_acc: 0.9219
Epoch 16/200
66/66 [==============================] - 203s 3s/step - l

Epoch 40/200
66/66 [==============================] - 204s 3s/step - loss: 0.6436 - acc: 0.9640 - val_loss: 0.5680 - val_acc: 0.9766
Epoch 41/200
66/66 [==============================] - 204s 3s/step - loss: 0.6476 - acc: 0.9527 - val_loss: 0.5683 - val_acc: 0.9844
Epoch 42/200
66/66 [==============================] - 204s 3s/step - loss: 0.6173 - acc: 0.9735 - val_loss: 0.5516 - val_acc: 0.9766
Epoch 43/200
66/66 [==============================] - 205s 3s/step - loss: 0.6354 - acc: 0.9773 - val_loss: 0.5956 - val_acc: 0.9766
Epoch 44/200
66/66 [==============================] - 204s 3s/step - loss: 0.6424 - acc: 0.9659 - val_loss: 0.5749 - val_acc: 0.9688
Epoch 45/200
66/66 [==============================] - 204s 3s/step - loss: 0.6615 - acc: 0.9564 - val_loss: 0.5644 - val_acc: 0.9688
Epoch 46/200
66/66 [==============================] - 204s 3s/step - loss: 0.6479 - acc: 0.9640 - val_loss: 0.6343 - val_acc: 0.9375
Epoch 47/200
66/66 [==============================] - 204s 3s/step - 

In [6]:
test_pred

array([[  5.02144632e-11,   1.68444008e-05,   1.24929461e-06, ...,
          2.60466611e-10,   2.42799421e-08,   8.17874746e-08],
       [  1.02301290e-07,   1.29256962e-04,   3.75487085e-05, ...,
          6.52675740e-08,   1.85423571e-06,   9.42297262e-08],
       [  2.11103907e-05,   6.27645123e-04,   9.59709525e-01, ...,
          1.54709763e-06,   1.16135895e-06,   1.53431431e-05],
       ..., 
       [  9.99663341e-01,   2.73752899e-08,   1.97968924e-05, ...,
          5.76375622e-07,   1.48610395e-08,   3.39200863e-06],
       [  6.41156139e-04,   3.20117127e-03,   1.04118448e-04, ...,
          3.70596951e-05,   6.04837896e-06,   3.40983289e-04],
       [  1.23120831e-03,   4.46316041e-03,   2.64275972e-04, ...,
          4.83353822e-05,   5.36010222e-05,   7.03687258e-03]])

In [7]:
predictions = np.argmax(test_pred, axis=1)

df = pd.DataFrame(predictions, columns=['Class'])
df['Id'] = df.index

# df.to_csv("submission.csv",index=False)

In [9]:
df

,Class,Id
0,4,0
1,16,1
2,2,2
3,18,3
4,11,4
5,5,5
6,6,6
7,15,7
8,8,8
9,8,9


In [8]:
df.to_csv("submission.csv",index=False)